In [1]:
#import dependencies

import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import balanced_accuracy_score, classification_report
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler, LabelEncoder

In [2]:
#Read in csv files, create dataframes

tuition_df = pd.read_csv('Resources\College_Pricing_Trends_76-24.csv')
underemployed_df = pd.read_csv('Resources\\Underemployed.csv')
unemployed_df = pd.read_csv('Resources\\Unemployed.csv')
wages_df = pd.read_csv('Resources\Wages.csv')


In [3]:
# Create function to drop unwanted columns that contain the same string

def drop_colums(df, column_contains):
    df = df.loc[:, ~df.columns.str.contains(column_contains)]
    return df

In [4]:
# Apply function

underemployed_df = drop_colums(underemployed_df, 'Unnamed')
wages_df = drop_colums(wages_df, 'Unnamed')

In [5]:
# Create function to convert date columns to desired 'Year' format

def convert_date(df):    
    df['Date'] = pd.to_datetime(df['Date'])
    df_august = df[df['Date'].dt.month == 8]
    df_august = df_august.reset_index(drop=True)
    df_august['Date'] = df_august['Date'].dt.year
    df_auguest = df_august.rename(columns={'Date':'Year'})
    return df_august


In [6]:
# Run dfs through function

unemployed_df_clean = convert_date(unemployed_df)
underemployed_df_clean = convert_date(underemployed_df)


C:\Users\clayt\AppData\Local\Temp\ipykernel_14872\3037807325.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Date'] = pd.to_datetime(df['Date'])
C:\Users\clayt\AppData\Local\Temp\ipykernel_14872\3037807325.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Date'] = pd.to_datetime(df['Date'])


In [7]:
# Convert 'Date' column in wages_df

wages_df['Date'] = pd.to_datetime(wages_df['Date'])
wages_df['Date'] = wages_df['Date'].dt.year

C:\Users\clayt\AppData\Local\Temp\ipykernel_14872\658603101.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  wages_df['Date'] = pd.to_datetime(wages_df['Date'])


In [8]:
# Convert 'Date Range' column in tuition_df

tuition_df['Academic Year'] = tuition_df['Academic Year'].apply(lambda x: '20' + x.split('-')[0] if int(x.split('-')[0]) < 50 else '19' + x.split('-')[0])
# Rename 'Acedemic Year' column to 'Year'

tuition_df = tuition_df.rename(columns={'Academic Year':'Year'})

# Display result
tuition_df

,Year,Private Nonprofit Four-Year,One-Year % Change,Public Four-Year,One-Year % Change.1,Public Two-Year,One-Year % Change.2,Private Nonprofit Four-Year.1,One-Year % Change.3,Public Four-Year.1,One-Year % Change.4
0,1971,"$13,820",—,"$3,250",—,"$1,430",—,"$22,130",—,"$10,650",—
1,1972,"$14,270",3.30%,"$3,660",12.60%,"$1,680",17.50%,"$22,610",2.20%,"$11,340",6.50%
2,1973,"$14,120",-1.10%,"$3,510",-4.10%,"$1,860",10.70%,"$22,180",-1.90%,"$11,020",-2.80%
3,1974,"$13,210",-6.40%,"$3,160",-10.00%,"$1,740",-6.50%,"$21,220",-4.30%,"$10,240",-7.10%
4,1975,"$13,020",-1.40%,"$3,070",-2.80%,"$1,420",-18.40%,"$20,920",-1.40%,"$10,120",-1.20%
5,1976,"$13,600",4.50%,"$3,330",8.50%,"$1,500",5.60%,"$21,390",2.20%,"$10,430",3.10%
6,1977,"$13,630",0.20%,"$3,330",0.00%,"$1,560",4.00%,"$21,400",0.00%,"$10,300",-1.20%
7,1978,"$13,880",1.80%,"$3,240",-2.70%,"$1,550",-0.60%,"$21,620",1.00%,"$10,080",-2.10%
8,1979,"$13,610",-1.90%,"$3,120",-3.70%,"$1,520",-1.90%,"$21,100",-2.40%,"$9,820",-2.60%
9,1980,"$13,440",-1.20%,"$2,970",-4.80%,"$1,450",-4.60%,"$20,750",-1.70%,"$9,460",-3.70%


In [9]:
# Rename 'Date' to 'Year'

unemployed_df_clean = unemployed_df_clean.rename(columns={'Date':'Year'})
underemployed_df_clean = underemployed_df_clean.rename(columns={'Date':'Year'})
wages_df_clean = wages_df.rename(columns={'Date':'Year'})


In [10]:
display(underemployed_df_clean.head())
display(unemployed_df_clean.head())
display(wages_df_clean.head())

,Year,Recent graduates,College graduates
0,1990,43.8,33.5
1,1991,44.4,33.9
2,1992,47.3,34.7
3,1993,46.8,34.4
4,1994,46.5,33.7


,Year,Young workers,All workers,Recent graduates,College graduates
0,1990,7.9,5.3,3.7,2.4
1,1991,9.9,6.6,4.8,3.0
2,1992,11.0,7.4,4.9,3.3
3,1993,10.1,6.6,4.0,2.9
4,1994,9.0,5.8,4.3,2.7


,Year,Bachelor's degree: 25th percentile,Bachelor's degree: median,Bachelor's degree: 75th percentile,High school diploma: median
0,1990,"39,984","52,353","67,188","38,073"
1,1991,"37,393","49,427","64,470","35,673"
2,1992,"37,540","50,054","62,567","33,369"
3,1993,"36,457","48,610","62,788","34,230"
4,1994,"35,387","47,380","61,199","33,561"


In [ ]:
#merged_df = df1.merge(df2, on='Key').merge(df3, on='Key').merge(df4, on='Key')